# S07 — Safe Pick-Up Spots & Safety Geofences

Генерируем **безопасные точки посадки** рядом с опасными зонами и строим **геозоны**
на основе высоких значений SRI.

In [ ]:
%run ./S00_setup.ipynb

In [ ]:
# 1) Загрузка SRI (edge) и hex features, если есть
sri_edge = pd.read_parquet(SRI_EDGE_PARQUET) if SRI_EDGE_PARQUET.exists() else None
edge_df = pd.read_parquet(EDGE_FEATURES_PARQUET) if EDGE_FEATURES_PARQUET.exists() else None
hex_df = pd.read_parquet(HEX_FEATURES_PARQUET) if HEX_FEATURES_PARQUET.exists() else None
print([e.shape if e is not None else None for e in [sri_edge, edge_df, hex_df]])
if sri_edge is None:
    raise SystemExit("Не найден SRI_EDGE. Запустите S06.")

In [ ]:
# 2) Геозоны: берём рёбра с SRI >= порога и строим буферы (упрощённо без геометрии используем точки из POINT_FEATURES)
THR = np.nanpercentile(sri_edge["SRI"], 85)  # топ-15% как опасные
sri_edge["is_hot"] = sri_edge["SRI"] >= THR
print("Hot edges:", sri_edge["is_hot"].mean())

# Если есть matched точки — используем их как геом.представление для простого геоjson (центры кластеров)
if MATCHED_PARQUET.exists():
    mm = pd.read_parquet(MATCHED_PARQUET)
    mm_join = mm.merge(sri_edge[["u","v","key","is_hot","SRI"]], on=["u","v","key"], how="inner")
    hot_pts = mm_join[mm_join["is_hot"]][["lat","lng","SRI"]].copy()
else:
    # Fallback: центры hex из HEX_FEATURES (требуется h3)
    hot_pts = None

# Экспорт упрощенного GeoJSON геозон как набор точек
features = []
if hot_pts is not None:
    for _,r in hot_pts.sample(min(len(hot_pts), 5000), random_state=42).iterrows():
        features.append({
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[float(r["lng"]), float(r["lat"])]},
            "properties":{"SRI": float(r["SRI"])}
        })
    geo = {"type":"FeatureCollection","features":features}
    with open(GEOFENCES_GEOJSON, "w", encoding="utf-8") as f:
        json.dump(geo, f)
    print("Saved geofences (points proxy):", GEOFENCES_GEOJSON)
else:
    print("Не удалось сформировать геозоны (нет MATCHED и H3).")

In [ ]:
# 3) Safe Pick-Up генератор: выбираем кандидатов в радиусе 150 м от опасных точек,
# среди участков с низким SRI (или низкой congestion), затем greedy покрытие.
MAX_CANDIDATES = 2000
RADIUS_M = 120.0

if MATCHED_PARQUET.exists():
    mm = pd.read_parquet(MATCHED_PARQUET)
    # Признаки безопасности кандидата — низкая congestion/низкий SRI (если доступен)
    base = mm.merge(sri_edge[["u","v","key","SRI"]], on=["u","v","key"], how="left")
    base = base.dropna(subset=["SRI"])
    # Нормируем удобство: score = - alpha*SRI - beta*freeflow (если есть) + gamma* (доля стопов низкая)
    cand = base[["lat","lng","SRI"]].copy()
    cand["score"] = -0.8*cand["SRI"]
    # Оставим топ кандидатов по score
    cand = cand.nsmallest(MAX_CANDIDATES, "score").reset_index(drop=True)

    # Покрытие опасных точек
    if hot_pts is not None and len(hot_pts) > 0:
        import numpy as np
        EARTH_R = 6_371_000.0
        hot = hot_pts[["lat","lng","SRI"]].copy()

        # Пространственный индекс (быстро) или безопасный fallback без sklearn
        try:
            from sklearn.neighbors import BallTree
            cand_rad = np.radians(cand[["lat","lng"]].values)
            hot_rad  = np.radians(hot[["lat","lng"]].values)
            tree = BallTree(hot_rad, metric="haversine")
            neighbors = tree.query_radius(cand_rad, r=RADIUS_M / EARTH_R)  # list[np.ndarray] на кандидата
        except Exception:
            # Без sklearn: предфильтруем по широте + векторная гаверсина по окну
            cand_xy = cand[["lat","lng"]].to_numpy()
            hot_xy  = hot[["lat","lng"]].to_numpy()
            lat_tol = RADIUS_M / 111_000.0
            order = np.argsort(hot_xy[:,0])
            hot_sorted = hot_xy[order]
            neighbors = []
            for lat, lng in cand_xy:
                lo, hi = lat - lat_tol, lat + lat_tol
                i0 = np.searchsorted(hot_sorted[:,0], lo)
                i1 = np.searchsorted(hot_sorted[:,0], hi)
                if i1 <= i0:
                    neighbors.append(np.array([], dtype=int))
                    continue
                clat, clng = np.radians([lat, lng])
                block = np.radians(hot_sorted[i0:i1])
                dlat = block[:,0] - clat
                dlng = block[:,1] - clng
                a = np.sin(dlat/2)**2 + np.cos(clat)*np.cos(block[:,0])*np.sin(dlng/2)**2
                dist = 2*EARTH_R*np.arcsin(np.sqrt(a))
                idx = order[i0:i1][dist <= RADIUS_M]
                neighbors.append(idx)

        covered = np.zeros(len(hot), dtype=bool)
        chosen = []
        for _ in range(min(10, len(cand))):
            # прирост покрытия по каждому кандидату (без iterrows)
            gains = np.fromiter(((~covered[idxs]).sum() for idxs in neighbors), dtype=int, count=len(neighbors))
            best = int(gains.argmax())
            if gains[best] <= 0:
                break
            chosen.append(cand.loc[best, ["lat","lng","score"]].to_dict())
            covered[neighbors[best]] = True
            neighbors[best] = np.array([], dtype=int)

        # Экспорт
        features = [{
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[float(c["lng"]), float(c["lat"])]},
            "properties":{"score": float(c["score"])}
        } for c in chosen]
        geo = {"type":"FeatureCollection","features":features}
        with open(SAFE_PU_GEOJSON, "w", encoding="utf-8") as f:
            json.dump(geo, f)
        print("Saved safe pick-ups:", SAFE_PU_GEOJSON)